- verl
    - data_preprocess
        - 封装数据 dataproto
    - generation
        - RLHFDataset
            - prompt_key: default is `prompt` (cite data_preprocess)
            - `tokenizer.apply_chat_template(doc[prompt_key], add_generation_prompt=True)`
    - reward_score/
- vLLMRollout => gen_batch_padded
    - idx.shape: `(, 256)`
    - response.shape: `(, 512)`
    - `input_ids = torch.cat([idx, response], dim=-1)`

```py
batch = TensorDict(
    {
        'prompts': idx,
        'responses': response,
        'input_ids': seq,  # here input_ids become the whole sentences
        # 'old_log_probs': log_probs, # we will recompute old log prob with actor
        'attention_mask': attention_mask,
        'position_ids': position_ids
    },
    batch_size=batch_size)
```

In [1]:
from transformers import AutoTokenizer
import re
import torch
from vllm import LLM, SamplingParams

/home/whaow/anaconda3/envs/verl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-30 23:40:35,506	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/whaow/anaconda3/envs/verl/lib/python3.9/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


In [2]:
base_id = 'Qwen/Qwen2.5-1.5B'
instruct_id = 'Qwen/Qwen2.5-1.5B-Instruct'

In [3]:
base_T = AutoTokenizer.from_pretrained(base_id)
instruct_T = AutoTokenizer.from_pretrained(instruct_id)

### inference

In [4]:
base_llm = LLM(model=base_id, max_model_len=8*1024)
# instruct_llm = LLM(model=instruct_id, max_model_len=8*1024)

INFO 03-30 23:40:55 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='Qwen/Qwen2.5-1.5B', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-1.5B, use_v2_block_manager=True, num_scheduler_steps=1, chunked_prefill_enabled=False multi_step_stream_outputs=True, enable_pr

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.62it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.62it/s]



INFO 03-30 23:40:59 model_runner.py:1071] Loading model weights took 2.9104 GB
INFO 03-30 23:41:00 gpu_executor.py:122] # GPU blocks: 38499, # CPU blocks: 9362
INFO 03-30 23:41:00 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 75.19x
INFO 03-30 23:41:04 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-30 23:41:04 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-30 23:41:13 model_runner.py:1530] Graph capturing finished in 10 secs.


In [5]:
target = 98
numbers = [44, 19, 35]

In [6]:
prefix = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>"""

In [7]:
print(prefix)

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers [44, 19, 35], create an equation that equals 98. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>


In [8]:
input_ids = base_T(prefix, return_tensors='pt', add_special_tokens=False)['input_ids']

In [9]:
sampling_params = SamplingParams(
        temperature=0.6, max_tokens=4*1024)
resp = base_llm.generate(prompt_token_ids=input_ids.squeeze().tolist(), sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  2.00s/it, est. speed input: 71.25 toks/s, output: 190.15 toks/s]


In [15]:
len(input_ids.squeeze().tolist())

142

In [22]:
print(base_T.decode(resp[0].prompt_token_ids))

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers [44, 19, 35], create an equation that equals 98. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>


In [23]:
print(resp[0].outputs[0].text)

 We need to create an equation using the numbers [44, 19, 35] and basic arithmetic operations (+, -, *, /) such that the equation equals 98. We can use each number only once. </think>

1. Let's start by taking the largest number, 44, and see if we can use it to get close to 98.
2. If we subtract 35 from 44, we get 9.
3. Now, we have 19 left. If we divide 9 by 19, we get a decimal number (0.4736842105263158).
4. Since we can't use a decimal number directly, let's try a different approach.
5. Let's try adding 19 to 19, which gives us 38.
6. Now, we need to find a way to get 60 (98 - 38) using the remaining numbers.
7. If we multiply 44 by 35, we get 1540.
8. We can divide 1540 by 23 (the result of dividing 19 by 19) to get 66.
9. Now, we have 66 left.
10. If we divide 66 by 1, we get 66.
11. Finally, we can divide 66 by 1 to get the result, which is 98.

Therefore, the equation that equals 98 using the numbers [44, 19, 35] and basic arithmetic operations is <answer> (44 - 35) / (19 * 19)

### chat_template

In [12]:
base_T.special_tokens_map

{'eos_token': '<|endoftext|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [13]:
print(base_T.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}
    {%- endif %}
{%- endif %}

In [14]:
print(instruct_T.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C